In [2]:
%pylab inline

import numpy as np
from sklearn.tree import DecisionTreeRegressor
from copy import deepcopy

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [3]:
class LambdaMART:
    def __init__(self, n_estimators, base_estimator=DecisionTreeRegressor(), step_estimator=DecisionTreeRegressor(),
                 alpha=1., beta=1., adaptive_step=True, feature_subset=False, feature_fraction=1., stochastic=False):
        """
            n_estimators : число деревьев в обучении
            base_estimator : выбор модели для каждой итерации
            step_estimator : выбор модели для предсказания темпа обучения
            alpha : коэффициент регуляризации XGBoost
            beta : коэффициент при предсказании темпа обучения
            adaptive_step : использовать адаптивный шаг бустинга или нет
            feature_subset : использовать случайный набор признаков или нет
        """
        self.estimators = []
        self.step_estimators = []
        self.feature_idx = []
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.step_estimator = step_estimator
        self.alpha = alpha
        self.beta = beta
        self.adaptive_step = adaptive_step
        self.feature_subset = feature_subset
        self.feature_fraction = feature_fraction
        self.stochastic = stochastic
        
    def fit(self, X_train, y_train, qids_train, queries_train, X_test, y_test, qids_test, queries_test, verbose=False):
        """
            X : признаки пар запрос-документ
            y : метки релевантности
            qids : id запросов
            queries : признаки запросов
        """
        X = X_train
        y = y_train
        qids = qids_train
        queries = queries_train
        self.estimators = []
        self.step_estimators = []
        # Препроцессинг для ускорения подсчитывания градиента
        id_y = np.argsort(np.argsort(y_train)[::-1])
        for i in xrange(self.n_estimators):
            if i % 30 == 1:
                y_pred = self.predict(X_test, qids_test, queries_test)
                print "Iteration ", i, " result ", ndcgl(y_pred, y_test)
            #print "Iteration #", i
            predicted = self.predict(X_train, qids_train, queries_train, verbose)
            sub_idx = []
            if self.stochastic:
                # Выбираем подвыборку элементов
                sub_idx = np.random.choice(np.arange(X_train.shape[0]), int(0.6 * X_train.shape[0]), replace=False)
                grad, h = self.loss_grad(predicted[sub_idx], y_train[sub_idx], id_y[sub_idx])
                X = X_train[sub_idx]
                y = y_train[sub_idx]
                qids = qids_train[sub_idx]
                queries = queries_train[sub_idx]
            else:
                grad, h = self.loss_grad(predicted, y, id_y)
            if verbose:
                print "Grad:"
                print grad
                print "H:"
                print h
            estimator = deepcopy(self.base_estimator)
            if self.feature_subset:
                # Список индексов фичей
                feature_idx = np.random.choice(np.arange(X.shape[1]), int(self.feature_fraction * X.shape[1]), replace=False)
                self.feature_idx.append(feature_idx)
                estimator.fit(X.T[feature_idx].T, -grad/ (self.alpha + h))
            else:
                estimator.fit(X, -grad/ (self.alpha + h))
            self.estimators.append(estimator)
            if verbose:
                print "Predict:"
                print predicted
            # Обучим предсказатель шага
            if self.adaptive_step:
                step_estimator = deepcopy(self.step_estimator)
                # Список всех запросов(уникальных)
                qs = np.unique(qids)
                # Список значений того, что нужно предсказывать
                pred_values = np.zeros(len(qs))
                q_list = np.zeros((len(qs), queries.shape[1]))
                for idx, q in enumerate(qs):
                    # Суммируем значения по всем элементам с данным запросом q
                    pred_values[idx] = 0.
                    for j in xrange(X.shape[0]):
                        if qids[j] == q:
                            q_list[idx] = queries[j]
                            pred_values[idx] += 1. / (1. + self.beta * h[j])            
                    #q_list[idx] = q_dict[q]
                sum_pred_values = np.sum(pred_values)
                step_estimator.fit(q_list, pred_values * len(qs) / sum_pred_values)
                self.step_estimators.append(step_estimator)
        return self
            
    def predict(self, X, qids, queries, verbose=False):
        y_pred = np.zeros(X.shape[0])
        for est_i, estimator in enumerate(self.estimators):
            if self.feature_subset:
                feature_idx = self.feature_idx[est_i]
                est_result = estimator.predict(X.T[feature_idx].T)
            else:
                est_result = estimator.predict(X)
            for i in xrange(X.shape[0]):
                step = 1. / (i + 1.)
                if self.adaptive_step:
                    step = self.step_estimators[est_i].predict(queries[i].reshape(1,-1)[0])
                if verbose:
                    print "Step: ", step, " for qid=", qids[i]
                y_pred[i] += est_result[i] * step
        return y_pred
    
    def loss_grad(self, pred_y, y, id_y):
        n_elems = y.shape[0]
        grad = np.zeros(n_elems)
        h = np.zeros(n_elems)
        # id_y - индексы в массиве pred_y, отсортированные по убыванию ранжирующей функции 
        #id_y = np.argsort(np.argsort(y)[::-1])
        
        for i in xrange(n_elems):
            for j in xrange(n_elems):
                buf = 0.
                delta_ndcg = np.abs(self.ndcg_replace(y[i], y[j], id_y[i], id_y[j]))
                if y[i] > y[j]:
                    buf = -self.rho(pred_y[i], pred_y[j])
                if y[i] < y[j]:
                    buf = self.rho(pred_y[j], pred_y[i])
                if buf != 0.:
                    grad[i] += delta_ndcg * buf
                h[i] += delta_ndcg * self.rho(pred_y[i], pred_y[j]) * (1 - self.rho(pred_y[i], pred_y[j]))
        return grad, h
        
    def double_grad(self, pred_y, y):
        n_elems = y.shape[0]
        h = np.zeros(n_elems)
        # id_y - индексы в массиве pred_y, отсортированные по убыванию ранжирующей функции 
        id_y = np.argsort(np.argsort(y)[::-1])
        for i in xrange(n_elems):
            for j in xrange(n_elems):
                delta_ndcg = np.abs(self.ndcg_replace(y[i], y[j], id_y[i], id_y[j]))
                h[i] += delta_ndcg * self.rho(pred_y[i], pred_y[j]) * (1 - self.rho(pred_y[i], pred_y[j]))
        return h
    
    def ndcg_replace(self, value_a, value_b, id_a, id_b):
        return (2. ** value_b - 2. ** value_a) * (1./ np.log2(2 + id_a) - 1./ np.log2(2 + id_b))
    
    def rho(self, a, b):
        return 1. / (1. + np.exp(a - b))

Считаем часть датасета MQ2007

In [4]:
# Проверка на части датасета MQ2007
fin = open('train.txt', 'r')

num_elems = 2000
X = np.zeros((num_elems, 46))
y = np.zeros(num_elems)
queries = np.zeros(num_elems)

num = 0
i = 0

lines = fin.readlines()[:num_elems]

for i, line in enumerate(lines):
    parsed = line.split(' ')
    y[i] = int(parsed[0])
    queries[i] = int(parsed[1][4:])
    for j in xrange(46):
        if j < 9:
            X[i][j] = float(parsed[j + 2][2:])
        else:
            X[i][j] = float(parsed[j + 2][3:])
                
# Нормализуем y
y /= np.max(y)

Генерируем признаки для запросов

In [5]:
from collections import defaultdict

# Признаки для запросов
query_d = 20 # Число признаков в запросе. 
q_dict = defaultdict(lambda : np.zeros(query_d)) # dictionary, который по id запроса возвращает его признаки

unique_q, counts = np.unique(queries, return_counts=True)
for i,q in enumerate(unique_q):
    for j in xrange(X.shape[0]):
        if queries[j] == q:
            q_dict[q] += X[j][11:31]/ counts[i]

# Финальная матрица для запросов. По индексу получаем признаки запроса, соответствующие элементу из X с этим индексом
q = np.zeros((num_elems, query_d))
for i in xrange(num_elems):
    q[i] = q_dict[queries[i]]

In [6]:
def ndcgl(y_pred, y, L=10):
    """
        y_pred : предсказанные значения функции ранжирования
        y : метки релевантности
    """
    dcgl = 0.
    idcgl = 0.
    # Отсортируем значения по убыванию функции ранжирования
    idx = np.argsort(y_pred)[::-1]
    # Метки релевантности для отсортированного массива y_pred
    l = y[idx]
    for i in xrange(L):
        dcgl += (2. ** l[i] - 1) / np.log2(i + 2)
        idcgl += 1 / np.log2(i + 2)
    return dcgl / idcgl

In [7]:
shuffle_idx = np.random.permutation(num_elems)
X_train = X[shuffle_idx][:num_elems/2.]
X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
X_test = X[shuffle_idx][num_elems*3./4.:]

y_train = y[shuffle_idx][:num_elems/2.]
y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
y_test = y[shuffle_idx][num_elems*3./4.:]

q_train = queries[shuffle_idx][:num_elems/2.]
q_valid = queries[shuffle_idx][num_elems/2.:num_elems*3./4.]
q_test = queries[shuffle_idx][num_elems*3./4.:]

queries_train = q[shuffle_idx][:num_elems/2.]
queries_valid = q[shuffle_idx][num_elems/2.:num_elems*3./4.]
queries_test = q[shuffle_idx][num_elems*3./4.:]

X_full_train = np.append(X_train, X_valid, axis=0)
y_full_train = np.append(y_train, y_valid)
q_full_train = np.append(q_train, q_valid)
queries_full_train = np.append(queries_train, queries_valid, axis=0)

In [44]:
%time clf1 = LambdaMART(20, alpha=0.5, beta=1., feature_subset=True, feature_fraction=0.3).fit(X_full_train, y_full_train, q_full_train, queries_full_train)

Wall time: 1min 1s


In [45]:
%time clf2 = LambdaMART(20, alpha=0.5, beta=1., feature_subset=True, \
                        feature_fraction=0.3, stochastic=True).fit(X_full_train, y_full_train, q_full_train, queries_full_train)

225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
Wall time: 25.3 s


In [47]:
print ndcgl(clf1.predict(X_test, q_test, queries_test), y_test)
print ndcgl(clf2.predict(X_test, q_test, queries_test), y_test)

0.586115924681
0.480710997422


In [16]:
%time DecisionTreeRegressor().fit(X_full_train, y_full_train)

Wall time: 71 ms


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [31]:
# Небольшой GridSearch с кросс-валидацией
from numpy import unravel_index

import warnings
warnings.filterwarnings('ignore')

arr_a = [0.5, 1., 2.] # Одно значение в целях оптимизации
arr_L = [1, 2, 5, 7]
arr_b = [0.5, 1., 2.] # Одно значение в целях оптимизации

results1 = np.zeros((len(arr_a), len(arr_L), len(arr_b))) # С адаптивным шагом
results2 = np.zeros((len(arr_a), len(arr_L), len(arr_b))) # Без адаптивного шага

for i_a, a in enumerate(arr_a):
    for i_L, L in enumerate(arr_L):
        for i_b, b in enumerate(arr_b):
            for k in xrange(5):
                # Перемешаем
                shuffle_idx = np.random.permutation(num_elems)
                X_train = X[shuffle_idx][:num_elems/2.]
                X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]

                y_train = y[shuffle_idx][:num_elems/2.]
                y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]

                q_train = queries[shuffle_idx][:num_elems/2.]
                q_valid = queries[shuffle_idx][num_elems/2.:num_elems*3./4.]

                queries_train = q[shuffle_idx][:num_elems/2.]
                queries_valid = q[shuffle_idx][num_elems/2.:num_elems*3./4.]
                
                clf1 = LambdaMART(L, alpha=a, beta=b).fit(X_train, y_train, q_train, queries_train)
                y_pred1 = clf1.predict(X_valid, q_valid, queries_valid)
                results1[i_a, i_L, i_b] += ndcgl(y_pred1, y_valid) / 5.

                clf2 = LambdaMART(L, alpha=a, beta=b, adaptive_step=False).fit(X_train, y_train, q_train, queries_train)
                y_pred2 = clf2.predict(X_valid, q_valid, queries_valid)
                results2[i_a, i_L, i_b] += ndcgl(y_pred2, y_valid) / 5.

                # Значения параметров и результаты с адаптивным шагом и без
            print a, L, b, results1[i_a, i_L, i_b], results2[i_a, i_L, i_b]

0.5 1 0.5 0.336099646865 0.334095124305
0.5 1 1.0 0.284810928463 0.336022586491
0.5 1 2.0 0.278983133131 0.235084867792
0.5 2 0.5 0.41276830698 0.325806649599
0.5 2 1.0 0.269452898252 0.219380711384
0.5 2 2.0 0.357579678573 0.328906145272
0.5 5 0.5 0.360515329939 0.291737452324
0.5 5 1.0 0.396132881545 0.352325317071
0.5 5 2.0 0.357491363348 0.307072058994
0.5 7 0.5 0.307196757893 0.243569464569
0.5 7 1.0 0.468726043507 0.381784322914
0.5 7 2.0 0.433379695178 0.35421116786
1.0 1 0.5 0.298903738381 0.329808291355
1.0 1 1.0 0.452061919927 0.348656410939
1.0 1 2.0 0.43510791712 0.279133197285
1.0 2 0.5 0.298245437817 0.270975813759
1.0 2 1.0 0.426176753028 0.298526360796
1.0 2 2.0 0.30197343755 0.296332263381
1.0 5 0.5 0.444450124696 0.334779000506
1.0 5 1.0 0.431277962121 0.373842327464
1.0 5 2.0 0.429456655432 0.344643182745
1.0 7 0.5 0.438494402494 0.358339454444
1.0 7 1.0 0.431007588023 0.367884660402
1.0 7 2.0 0.446516329361 0.336559264488
2.0 1 0.5 0.259869836501 0.210006019129
2.0 

In [32]:
ad_max = unravel_index(np.argmax(results1), results1.shape)
non_ad_max = unravel_index(np.argmax(results2), results2.shape)
        
print "Max with adaptive step - Alpha={}, L={}, beta={}".format(arr_a[ad_max[0]], arr_L[ad_max[1]], arr_b[ad_max[2]])
print "Max without adaptive step - Alpha={}, L={}, beta={}".format(arr_a[non_ad_max[0]], arr_L[non_ad_max[1]], arr_b[non_ad_max[2]])

print "Maximum with adaptive step=", np.max(results1)
print "Maximum without adaptive step=", np.max(results2)

Max with adaptive step - Alpha=0.5, L=7, beta=1.0
Max without adaptive step - Alpha=2.0, L=7, beta=2.0
Maximum with adaptive step= 0.468726043507
Maximum without adaptive step= 0.394820552599


В принципе, из списка на всех параметрах видны корреляции для адаптивного и неадаптивного методов. И как правило, увеличение числа итераций хорошо влияет на результат. Можно было продолжить и далее, но в виду ограниченных мощностей, пока это не главное.

In [44]:
result1 = 0.
result2 = 0.

for k in xrange(5):
    shuffle_idx = np.random.permutation(num_elems)
    X_train = X[shuffle_idx][:num_elems/2.]
    X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
    X_test = X[shuffle_idx][num_elems*3./4.:]

    y_train = y[shuffle_idx][:num_elems/2.]
    y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
    y_test = y[shuffle_idx][num_elems*3./4.:]

    q_train = queries[shuffle_idx][:num_elems/2.]
    q_valid = queries[shuffle_idx][num_elems/2.:num_elems*3./4.]
    q_test = queries[shuffle_idx][num_elems*3./4.:]

    queries_train = q[shuffle_idx][:num_elems/2.]
    queries_valid = q[shuffle_idx][num_elems/2.:num_elems*3./4.]
    queries_test = q[shuffle_idx][num_elems*3./4.:]
    
    X_full_train = np.append(X_train, X_valid, axis=0)
    y_full_train = np.append(y_train, y_valid)
    q_full_train = np.append(q_train, q_valid)
    queries_full_train = np.append(queries_train, queries_valid, axis=0)
    
    clf1 = LambdaMART(7, alpha=0.5, beta=1.).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
    y_pred1 = clf1.predict(X_test, q_test, queries_test)

    clf2 = LambdaMART(7, alpha=2.0, beta=2.0, adaptive_step=False).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
    y_pred2 = clf2.predict(X_test, q_test, queries_test)
    
    result1 += ndcgl(y_pred1, y_test) / 5.
    result2 += ndcgl(y_pred2, y_test) / 5.

print "Adaptive=", result1
print "Non-adaptive=", result2

Adaptive= 0.473884366146
Non-adaptive= 0.328375116338


In [34]:
# Обычный DecisionTreeRegressor
tempresult = 0.

for k in xrange(5):
    shuffle_idx = np.random.permutation(num_elems)
    X_train = X[shuffle_idx][:num_elems/2.]
    X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
    X_test = X[shuffle_idx][num_elems*3./4.:]

    y_train = y[shuffle_idx][:num_elems/2.]
    y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
    y_test = y[shuffle_idx][num_elems*3./4.:]
    
    X_full_train = np.append(X_train, X_valid, axis=0)
    y_full_train = np.append(y_train, y_valid)

    tempclf = DecisionTreeRegressor().fit(X_full_train, y_full_train)
    tempresult += ndcgl(tempclf.predict(X_test), y_test) / 5.
    
print tempresult

0.349995209906


In [43]:
# GDBT
from sklearn.ensemble import GradientBoostingRegressor

tempresult = 0.

for k in xrange(5):
    shuffle_idx = np.random.permutation(num_elems)
    X_train = X[shuffle_idx][:num_elems/2.]
    X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
    X_test = X[shuffle_idx][num_elems*3./4.:]

    y_train = y[shuffle_idx][:num_elems/2.]
    y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
    y_test = y[shuffle_idx][num_elems*3./4.:]
    
    X_full_train = np.append(X_train, X_valid, axis=0)
    y_full_train = np.append(y_train, y_valid)

    tempclf = GradientBoostingRegressor().fit(X_full_train, y_full_train)
    tempresult += ndcgl(tempclf.predict(X_test), y_test) / 5.
    
print tempresult

0.424421739725


### LambdaMART из XGBoost

In [13]:
# Необходимые технические строки. Исполняйте их в случае ошибки WindowsError Error 127. 
# Путь заменить на своё расположение mingw-64
dir = r'C:\Program Files\mingw-w64\x86_64-6.2.0-posix-seh-rt_v5-rev1\mingw64\bin'
import os

os.environ['PATH'].find(dir)
os.environ['PATH'] = dir + ';' + os.environ['PATH']

In [14]:
import xgboost as xgb

In [40]:
tempresult = 0.

for k in xrange(5):
    shuffle_idx = np.random.permutation(num_elems)
    X_train = X[shuffle_idx][:num_elems/2.]
    X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
    X_test = X[shuffle_idx][num_elems*3./4.:]

    y_train = y[shuffle_idx][:num_elems/2.]
    y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
    y_test = y[shuffle_idx][num_elems*3./4.:]
    
    X_full_train = np.append(X_train, X_valid, axis=0)
    y_full_train = np.append(y_train, y_valid)
    
    clf = xgb.XGBClassifier(objective='rank:ndcg')
    clf.fit(X_full_train,y_full_train)
    tempresult += ndcgl(clf.predict(X_test), y_test) / 5.
    
print tempresult

0.434840804088


### Работа на 4000 элементах, 70 итераций бустинга

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
shuffle_idx = np.random.permutation(num_elems)
X_train = X[shuffle_idx][:num_elems/2.]
X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
X_test = X[shuffle_idx][num_elems*3./4.:]

y_train = y[shuffle_idx][:num_elems/2.]
y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
y_test = y[shuffle_idx][num_elems*3./4.:]

q_train = queries[shuffle_idx][:num_elems/2.]
q_valid = queries[shuffle_idx][num_elems/2.:num_elems*3./4.]
q_test = queries[shuffle_idx][num_elems*3./4.:]

queries_train = q[shuffle_idx][:num_elems/2.]
queries_valid = q[shuffle_idx][num_elems/2.:num_elems*3./4.]
queries_test = q[shuffle_idx][num_elems*3./4.:]

X_full_train = np.append(X_train, X_valid, axis=0)
y_full_train = np.append(y_train, y_valid)
q_full_train = np.append(q_train, q_valid)
queries_full_train = np.append(queries_train, queries_valid, axis=0)

%time clf_big = LambdaMART(70, alpha=0.5, beta=1., ).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
#y_pred1 = clf1.predict(X_test, q_test, queries_test)

Iteration # 0
Iteration # 1
Iteration # 2
Iteration # 3
Iteration # 4
Iteration # 5
Iteration # 6
Iteration # 7
Iteration # 8
Iteration # 9
Iteration # 10
Iteration # 11
Iteration # 12
Iteration # 13
Iteration # 14
Iteration # 15
Iteration # 16
Iteration # 17
Iteration # 18
Iteration # 19
Iteration # 20
Iteration # 21
Iteration # 22
Iteration # 23
Iteration # 24
Iteration # 25
Iteration # 26
Iteration # 27
Iteration # 28
Iteration # 29
Iteration # 30
Iteration # 31
Iteration # 32
Iteration # 33
Iteration # 34
Iteration # 35
Iteration # 36
Iteration # 37
Iteration # 38
Iteration # 39
Iteration # 40
Iteration # 41
Iteration # 42
Iteration # 43
Iteration # 44
Iteration # 45
Iteration # 46
Iteration # 47
Iteration # 48
Iteration # 49
Iteration # 50
Iteration # 51
Iteration # 52
Iteration # 53
Iteration # 54
Iteration # 55
Iteration # 56
Iteration # 57
Iteration # 58
Iteration # 59
Iteration # 60
Iteration # 61
Iteration # 62
Iteration # 63
Iteration # 64
Iteration # 65
Iteration # 66
Itera

In [12]:
y_pred_big = clf_big.predict(X_test, q_test, queries_test)
print ndcgl(y_pred_big, y_test)

0.896477581668


In [16]:
# xgboost
clf = xgb.XGBClassifier(objective='rank:ndcg')
clf.fit(X_full_train,y_full_train)
print ndcgl(clf.predict(X_test), y_test)

0.733113475428


### Проверка случайного выбора фичей

In [7]:
shuffle_idx = np.random.permutation(num_elems)
X_train = X[shuffle_idx][:num_elems/2.]
X_valid = X[shuffle_idx][num_elems/2.:num_elems*3./4.]
X_test = X[shuffle_idx][num_elems*3./4.:]

y_train = y[shuffle_idx][:num_elems/2.]
y_valid = y[shuffle_idx][num_elems/2.:num_elems*3./4.]
y_test = y[shuffle_idx][num_elems*3./4.:]

q_train = queries[shuffle_idx][:num_elems/2.]
q_valid = queries[shuffle_idx][num_elems/2.:num_elems*3./4.]
q_test = queries[shuffle_idx][num_elems*3./4.:]

queries_train = q[shuffle_idx][:num_elems/2.]
queries_valid = q[shuffle_idx][num_elems/2.:num_elems*3./4.]
queries_test = q[shuffle_idx][num_elems*3./4.:]

X_full_train = np.append(X_train, X_valid, axis=0)
y_full_train = np.append(y_train, y_valid)
q_full_train = np.append(q_train, q_valid)
queries_full_train = np.append(queries_train, queries_valid, axis=0)

results = []

for n in xrange(10, 600, 30):
    print n

    clf1 = LambdaMART(n, alpha=0.5, beta=1.).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
    clf2 = LambdaMART(n, alpha=0.5, beta=1., feature_subset=True, feature_fraction=0.3).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
    clf3 = LambdaMART(n, alpha=0.5, beta=1., feature_subset=True, feature_fraction=0.3, \
                      stochastic=True).fit(X_full_train, y_full_train, q_full_train, queries_full_train)
    
    result1 = ndcgl(clf1.predict(X_test, q_test, queries_test), y_test)
    result2 = ndcgl(clf2.predict(X_test, q_test, queries_test), y_test)
    result3 = ndcgl(clf3.predict(X_test, q_test, queries_test), y_test)
    results.append((result1, result2, result3))
    
    print "Without subset", result1
    print "With subset", result2
    print "Stochastic", result3

10
Without subset 0.589559508361
With subset 0.525202845018
Stochastic 0.465555208682
40
Without subset 0.670012899467
With subset 0.851643743565
Stochastic 0.79434887815
70
Without subset 0.595825485951
With subset 0.837945450273
Stochastic 0.732566107838
100
Without subset 0.625597894605
With subset 0.805930029937
Stochastic 0.621950881675
130
Without subset 0.635118472228
With subset 0.838777861945
Stochastic 0.646343802709
160


KeyboardInterrupt: 

In [14]:
print ndcgl(clf1.predict(X_test, q_test, queries_test), y_test)
print ndcgl(clf2.predict(X_test, q_test, queries_test), y_test)

0.619296498687
0.446995121205


In [9]:
results

[(0.58955950836136284, 0.52520284501832992, 0.46555520868212591),
 (0.67001289946652387, 0.85164374356483075, 0.79434887815035293),
 (0.59582548595062079, 0.83794545027274325, 0.73256610783800979),
 (0.62559789460458759, 0.80593002993736984, 0.6219508816753645),
 (0.63511847222811579, 0.83877786194521575, 0.64634380270875258)]